# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: Computer Systems Engineering  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: 31/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ParDeDosLogsStreaming") \
    .master("spark://b8beb2188649:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/03 04:36:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
log_dir="/home/jovyan/notebooks/data/structured_streaming_log_files/"

logs_lines = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "delete") \
                .load(log_dir)

In [5]:
from pyspark.sql.functions import split

logs_df = logs_lines.select(split(logs_lines.value, " | ").alias("log_array"))
logs_df = logs_df.withColumn("date_time", logs_df["log_array"].getItem(0).cast("string"))
logs_df = logs_df.withColumn("date", split(logs_df["date_time"], " ").getItem(0).cast("date"))
logs_df = logs_df.withColumn("time", split(logs_df["date_time"], " ").getItem(1).cast("timestamp"))
logs_df = logs_df.withColumn("log_level", logs_df["log_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("message", logs_df["log_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("node", logs_df["log_array"].getItem(3).cast("string"))

logs_df.printSchema()

root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date_time: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- log_level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [6]:
logs_df = logs_df.filter(logs_df.log_level == "ERROR")

In [7]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='10 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(60)

25/04/03 04:37:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a21b3009-0f32-4aca-9cbf-ef995d273d7b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/03 04:37:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+---------+----+----+---------+-------+----+
|log_array|date_time|date|time|log_level|message|node|
+---------+---------+----+----+---------+-------+----+
+---------+---------+----+----+---------+-------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---------+---------+----+----+---------+-------+----+
|log_array|date_time|date|time|log_level|message|node|
+---------+---------+----+----+---------+-------+----+
+---------+---------+----+----+---------+-------+----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+---------+----+----+---------+-------+----+
|log_array|date_time|date|time|log_level|message|node|
+---------+---------+----+----+---------+-------+----+
+---------+---------+----+----+---------+-------+----+

-------------------------------------------
Ba

False

In [8]:
query.stop()

In [9]:
sc.stop()